Relax Challenge Solution
This notebook provides my analysis and conclusions to the Relax Data Science Challenge. Here is the narrative that outlines the challenge:

The data is available as two attached CSV files:

takehome_user_engagement. csv

takehome_users . csv

The data has the following two tables:

1 - A user table ( "takehome_users" ) with data on 12,000 users who signed up for the product in the last two years. This table includes:

name: the user's name
object_id: the user's id
email: email address
creation_source: how their account was created. This takes on one of 5 values:
PERSONAL_PROJECTS: invited to join another user's personal workspace
GUEST_INVITE: invited to an organization as a guest (limited permissions)
ORG_INVITE: invited to an organization (as a full member)
SIGNUP: signed up via the website
SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id)
creation_time: when they created their account
last_session_creation_time: unix timestamp of last login
opted_in_to_mailing_list: whether they have opted into receiving marketing emails
enabled_for_marketing_drip: whether they are on the regular marketing email drip
org_id: the organization (group of users) they belong to
invited_by_user_id: which user invited them to join (if applicable).
2 - A usage summary table ( "takehome_user_engagement" ) that has a row for each day that a user logged into the product.

Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven-day period , identify which factors predict future user adoption.

We suggest spending 1-2 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better -- no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 500

%matplotlib inline

Load the two files into dataframes to see how they are organized.

In [4]:
df_users = pd.read_csv('takehome_users.csv', encoding='latin-1', index_col=0)
df_users

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0


In [5]:
df_eng = pd.read_csv('takehome_user_engagement.csv')
df_eng

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


First, I will check to see if df.eng has anything faulty, such as missing data or wrong datatypes.

In [7]:
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


We have a dataset containing 207,917 rows of user login records, with no missing data. The time_stamp column captures the date and time of each login but is currently stored as strings in the DataFrame. The next step will be to convert this column into a proper datetime format and use it as the DataFrame’s index for easier time-based analysis.

In [14]:
df_eng['time_stamp'] = pd.to_datetime(df_eng['time_stamp'])

In [16]:
df_eng = df_eng.set_index('time_stamp')
df_eng

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1
...,...,...
2013-09-06 06:14:15,11996,1
2013-01-15 18:28:37,11997,1
2014-04-27 12:45:16,11998,1


With the datetime index now in place, I can apply the .resample() method to aggregate the dates into weekly intervals. Although this approach may not perfectly align with every 7-day span in which a user adopts the product, it should capture the majority of users who log in at least three times a week, making it suitable for this initial analysis. I’ll start by grouping the data by user_id, then resample the dates by week and sum the visited column. This will allow us to identify which users can be classified as "adopted."

In [18]:
df_eng = df_eng.groupby('user_id').resample('1W').sum()
df_eng

user_id  visited
user_id time_stamp                  
1       2014-04-27        1        1
2       2013-11-17        2        1
        2013-11-24        0        0
        2013-12-01        2        1
        2013-12-08        0        0
...                     ...      ...
11996   2013-09-08    11996        1
11997   2013-01-20    11997        1
11998   2014-04-27    11998        1
11999   2012-06-03    11999        1
12000   2014-01-26    12000        1

[82282 rows x 2 columns]

Next, I will isolate those who have logged into the product atleast 3 times within one week and delete the extra user_id column.

In [23]:
df_adopt = df_eng[df_eng['visited'] > 2]
del df_adopt['user_id']
df_adopt

visited
user_id time_stamp         
2       2014-02-09        3
10      2013-03-03        3
        2013-04-14        3
        2013-04-28        3
        2013-05-05        4
...                     ...
11988   2014-04-06        3
        2014-05-04        4
        2014-05-18        3
        2014-05-25        5
        2014-06-01        3

[33859 rows x 1 columns]

We have 33,859 instances where users logged into the product more than 3 times in a week, though these entries don’t all represent unique users. To address this, I will generate a list of user IDs that meet the adoption criteria and use it to label the other dataset.

In [30]:
adopt_index = list(df_adopt.index)
adopt_users_list = []
for i in range(len(adopt_index)):
    adopt_users_list.append(adopt_index[i][0])
len(adopt_users_list)

33859

Remove any duplicates

In [33]:
adopt_users = set(adopt_users_list)
len(adopt_users)

1445

We’ve identified that 1,445 out of the 12,000 users have adopted the product, representing 12% of the total user base. Now, let’s update the df_users DataFrame by adding a label to indicate which users are adopted.

In [38]:
df_users['adopted'] = False
for user in adopt_users:
    df_users.loc[user, 'adopted'] = True

df_users.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,False
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,False
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,False
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,False


Now that the adopted user label has been added, I’ll review the df_users DataFrame to identify any required data cleaning and gain deeper insights into the dataset’s features.

In [43]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   name                        12000 non-null  object 
 2   email                       12000 non-null  object 
 3   creation_source             12000 non-null  object 
 4   last_session_creation_time  8823 non-null   float64
 5   opted_in_to_mailing_list    12000 non-null  int64  
 6   enabled_for_marketing_drip  12000 non-null  int64  
 7   org_id                      12000 non-null  int64  
 8   invited_by_user_id          6417 non-null   float64
 9   adopted                     12000 non-null  bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 1.2+ MB


It turns out that two columns in the dataset contain missing values, with the 'last_session_creation_time' missing data for nearly one-third of the users. I’ve decided to leave this feature out of the model. Since it likely correlates strongly with the adoption status (recent logins are probably tied to product adoption), including it would diminish the model's value. In short, building a model that simply confirms that recent activity indicates higher adoption wouldn’t provide much meaningful insight. 

Investigating the missing values in the 'invited_by_user_id'.

In [47]:
df_users['invited_by_user_id'].value_counts(ascending=False, dropna=False)

invited_by_user_id
NaN        5583
10741.0      13
2527.0       12
1525.0       11
2308.0       11
           ... 
2071.0        1
1390.0        1
5445.0        1
8526.0        1
5450.0        1
Name: count, Length: 2565, dtype: int64

The 'invited_by_user_id' feature seems to represent the ID of the user who invited a particular user, or a NaN value if no invitation occurred. I plan to convert this feature into a boolean format, indicating whether a user was invited by an existing user, for inclusion in the model.

In [50]:
df_users['invited_by_user'] = df_users['invited_by_user_id'].isnull() == False
del df_users['invited_by_user_id']
df_users.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,False,True
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,True,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,False,True
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,False,True
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,False,True
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,False,True
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,False,False
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,False,False
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,False,False


The dataset includes each user's individual email address, which isn't directly useful for the model. However, identifying the most common email domains among the users could provide valuable insights. I’ll extract the domain information from the email addresses for further analysis.

In [53]:
df_users['email_split'] = df_users['email'].str.split('@')
del df_users['email']

for user in range(len(df_users)):
    df_users.loc[user+1, 'email_domain'] = df_users.loc[user+1, 'email_split'][1]
del df_users['email_split']

df_users['email_domain'].value_counts()[:10]

email_domain
gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
hotmail.com       1165
rerwl.com            2
oqpze.com            2
qgjbc.com            2
dqwln.com            2
Name: count, dtype: int64

There appear to be six common email domains among the users. To simplify the data for modeling, I’ll group all the less common domains into a single 'Other' category, allowing us to use this as a categorical feature in the model.

In [56]:
common_domains = ['gmail.com', 'yahoo.com', 'jourrapide.com', 'cuvox.de', 'gustr.com', 'hotmail.com']

df_users['email'] = df_users['email_domain'].apply(lambda x: x if x in common_domains else 'other')

df_users.drop('email_domain', axis=1, inplace=True)

df_users['email'].value_counts(ascending=False)


email
gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
other             1186
gustr.com         1179
hotmail.com       1165
Name: count, dtype: int64

In [60]:
df_users.head()

,creation_time,name,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user,email
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,GUEST_INVITE,1.398139e+09,1,0,11,False,True,yahoo.com
2,2013-11-15 03:45:04,Poole Matthew,ORG_INVITE,1.396238e+09,0,0,1,True,True,gustr.com
3,2013-03-19 23:14:52,Bottrill Mitchell,ORG_INVITE,1.363735e+09,0,0,94,False,True,gustr.com
4,2013-05-21 08:09:28,Clausen Nicklas,GUEST_INVITE,1.369210e+09,0,0,1,False,True,yahoo.com
5,2013-01-17 10:14:20,Raw Grace,GUEST_INVITE,1.358850e+09,0,0,193,False,True,yahoo.com


The next step is to organize the data for building the predictive model. I’ll start by creating a new DataFrame, df_ml, where the 'adopted user' feature will serve as the target label in the first column, with 'user_id' set as the index.

In [63]:
df_ml = pd.DataFrame({'adopted_user': df_users['adopted'].values}, index=df_users.index)
df_ml

,adopted_user
object_id,
1,False
2,True
3,False
4,False
5,False
...,...
11996,False
11997,False
11998,False


Several binary categorical variables are already in the correct format and can be directly added to the DataFrame without any further modifications.

In [66]:
df_ml['mailing_list'] = df_users['opted_in_to_mailing_list']
df_ml['marketing_drip'] = df_users['enabled_for_marketing_drip']
df_ml['invited_by_user'] = df_users['invited_by_user']
df_ml

,adopted_user,mailing_list,marketing_drip,invited_by_user
object_id,,,,
1,False,1,0,True
2,True,0,0,True
3,False,0,0,True
4,False,0,0,True
5,False,0,0,True
...,...,...,...,...
11996,False,0,0,True
11997,False,0,0,False
11998,False,1,1,True


The dataset contains several categorical variables, which I will convert into dummy variables (one-hot encoding) and then append them to the df_ml DataFrame for use in the model.

In [69]:
df_users['org_id'] = df_users['org_id'].astype('str')

df_dummies = pd.get_dummies(df_users[['creation_source', 'org_id', 'email']])

df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head(3)

,adopted_user,mailing_list,marketing_drip,invited_by_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,org_id_1,org_id_10,org_id_100,org_id_101,org_id_102,org_id_103,org_id_104,org_id_105,org_id_106,org_id_107,org_id_108,org_id_109,org_id_11,org_id_110,org_id_111,org_id_112,org_id_113,org_id_114,org_id_115,org_id_116,org_id_117,org_id_118,org_id_119,org_id_12,org_id_120,org_id_121,org_id_122,org_id_123,org_id_124,org_id_125,org_id_126,org_id_127,org_id_128,org_id_129,org_id_13,org_id_130,org_id_131,org_id_132,org_id_133,org_id_134,org_id_135,org_id_136,org_id_137,org_id_138,org_id_139,org_id_14,org_id_140,org_id_141,org_id_142,org_id_143,org_id_144,org_id_145,org_id_146,org_id_147,org_id_148,org_id_149,org_id_15,org_id_150,org_id_151,org_id_152,org_id_153,org_id_154,org_id_155,org_id_156,org_id_157,org_id_158,org_id_159,org_id_16,org_id_160,org_id_161,org_id_162,org_id_163,org_id_164,org_id_165,org_id_166,org_id_167,org_id_168,org_id_169,org_id_17,org_id_170,org_id_171,org_id_172,org_id_173,org_id_174,org_id_175,org_id_176,org_id_177,org_id_178,org_id_179,org_id_18,org_id_180,org_id_181,org_id_182,org_id_183,org_id_184,org_id_185,org_id_186,org_id_187,org_id_188,org_id_189,org_id_19,org_id_190,org_id_191,org_id_192,org_id_193,org_id_194,org_id_195,org_id_196,org_id_197,org_id_198,org_id_199,org_id_2,org_id_20,org_id_200,org_id_201,org_id_202,org_id_203,org_id_204,org_id_205,org_id_206,org_id_207,org_id_208,org_id_209,org_id_21,org_id_210,org_id_211,org_id_212,org_id_213,org_id_214,org_id_215,org_id_216,org_id_217,org_id_218,org_id_219,org_id_22,org_id_220,org_id_221,org_id_222,org_id_223,org_id_224,org_id_225,org_id_226,org_id_227,org_id_228,org_id_229,org_id_23,org_id_230,org_id_231,org_id_232,org_id_233,org_id_234,org_id_235,org_id_236,org_id_237,org_id_238,org_id_239,org_id_24,org_id_240,org_id_241,org_id_242,org_id_243,org_id_244,org_id_245,org_id_246,org_id_247,org_id_248,org_id_249,org_id_25,org_id_250,org_id_251,org_id_252,org_id_253,org_id_254,org_id_255,org_id_256,org_id_257,org_id_258,org_id_259,org_id_26,org_id_260,org_id_261,org_id_262,org_id_263,org_id_264,org_id_265,org_id_266,org_id_267,org_id_268,org_id_269,org_id_27,org_id_270,org_id_271,org_id_272,org_id_273,org_id_274,org_id_275,org_id_276,org_id_277,org_id_278,org_id_279,org_id_28,org_id_280,org_id_281,org_id_282,org_id_283,org_id_284,org_id_285,org_id_286,org_id_287,org_id_288,org_id_289,org_id_29,org_id_290,org_id_291,org_id_292,org_id_293,org_id_294,org_id_295,org_id_296,org_id_297,org_id_298,org_id_299,org_id_3,org_id_30,org_id_300,org_id_301,org_id_302,org_id_303,org_id_304,org_id_305,org_id_306,org_id_307,org_id_308,org_id_309,org_id_31,org_id_310,org_id_311,org_id_312,org_id_313,org_id_314,org_id_315,org_id_316,org_id_317,org_id_318,org_id_319,org_id_32,org_id_320,org_id_321,org_id_322,org_id_323,org_id_324,org_id_325,org_id_326,org_id_327,org_id_328,org_id_329,org_id_33,org_id_330,org_id_331,org_id_332,org_id_333,org_id_334,org_id_335,org_id_336,org_id_337,org_id_338,org_id_339,org_id_34,org_id_340,org_id_341,org_id_342,org_id_343,org_id_344,org_id_345,org_id_346,org_id_347,org_id_348,org_id_349,org_id_35,org_id_350,org_id_351,org_id_352,org_id_353,org_id_354,org_id_355,org_id_356,org_id_357,org_id_358,org_id_359,org_id_36,org_id_360,org_id_361,org_id_362,org_id_363,org_id_364,org_id_365,org_id_366,org_id_367,org_id_368,org_id_369,org_id_37,org_id_370,org_id_371,org_id_372,org_id_373,org_id_374,org_id_375,org_id_376,org_id_377,org_id_378,org_id_379,org_id_38,org_id_380,org_id_381,org_id_382,org_id_383,org_id_384,org_id_385,org_id_386,org_id_387,org_id_388,org_id_389,org_id_39,org_id_390,org_id_391,org_id_392,org_id_393,org_id_394,org_id_395,org_id_396,org_id_397,org_id_398,org_id_399,org_id_4,org_id_40,org_id_400,org_id_401,org_id_402,org_id_403,org_id_404,org_id_405,org_id_406,org_id_407,org_id_408,org_id_409,org_id_41,org_id_410,org_id_4

Next, I’ll proceed with training a predictive model. According to the scikit-learn model selection flowchart, for classification problems with fewer than 100k samples, an ensemble model is recommended. I’ll use Random Forest as the ensemble classifier for this task.
The first step will be to split the data into training and test sets:

In [72]:
X = df_ml.iloc[:, 1:]
y = df_ml.loc[:, 'adopted_user']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

In [74]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=6)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print('Accuracy on training set = {}'.format(rfc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc.score(X_test, y_test)))

Accuracy on training set = 0.9426666666666667
Accuracy on test set = 0.836


In [76]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

       False       0.88      0.94      0.91      2639
        True       0.17      0.09      0.12       361

    accuracy                           0.84      3000
   macro avg       0.53      0.52      0.52      3000
weighted avg       0.80      0.84      0.81      3000



In [78]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc))

[[2474  165]
 [ 327   34]]


Next, I will create a dataframe of feature importance and sort by the highest value.

In [81]:
df_features = pd.DataFrame({'rfc': rfc.feature_importances_}, index=df_ml.columns[1:])

df_features.sort_values('rfc', ascending=False)[:15]

,rfc
mailing_list,0.035676
marketing_drip,0.024367
email_gmail.com,0.016870
email_jourrapide.com,0.013444
email_yahoo.com,0.013412
email_other,0.013091
email_gustr.com,0.012633
creation_source_PERSONAL_PROJECTS,0.011543
email_cuvox.de,0.009997
email_hotmail.com,0.009989


It seems that the "out-of-the-box" Random Forest Classifier performed well by correctly classifying 84% of the test set samples. However, it's crucial to remember that the null accuracy rate is 88%, meaning the model could have achieved similar accuracy by simply predicting that no users adopted the product. Given the imbalance in the dataset, where the majority of users did not adopt the product, it's more important to assess how well the model identifies the minority class—adopted users. Models tend to favor the majority class when data is imbalanced, which makes accurately detecting the minority class critical for this project. Since the goal is to determine which features are most predictive of future user adoption, focusing on this minority group is essential.

In this case, the classification report and confusion matrix reveal that the "out-of-the-box" model performed poorly at identifying adopted users, correctly classifying only 29 out of 361 in the test set. As a result, the feature importance values from this model will not provide meaningful insights into which features predict product adoption.

However, not all is lost. The Random Forest Classifier has several hyperparameters that can be fine-tuned to improve performance. Let’s adjust these hyperparameters to see if we can enhance the model’s ability to correctly classify adopted users.

In [84]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=6)
param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [5, 15, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
grid = GridSearchCV(rfc, param_grid, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
y_pred_rfc = grid.predict(X_test)
print(grid.score(X_test, y_test))

0.8796666666666667


In [86]:
grid.best_score_

0.8795555555555555

In [88]:
grid.best_estimator_

RandomForestClassifier(max_depth=5, random_state=6)

In [90]:
from sklearn.ensemble import RandomForestClassifier
rfc_tuned = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=1, criterion='gini', class_weight='balanced_subsample', random_state=6)
rfc_tuned.fit(X_train, y_train)
y_pred_rfc_tuned = rfc_tuned.predict(X_test)
print('Accuracy on training set = {}'.format(rfc_tuned.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc_tuned.score(X_test, y_test)))

Accuracy on training set = 0.6205555555555555
Accuracy on test set = 0.6


In [92]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc_tuned))

              precision    recall  f1-score   support

       False       0.90      0.61      0.73      2639
        True       0.15      0.51      0.23       361

    accuracy                           0.60      3000
   macro avg       0.53      0.56      0.48      3000
weighted avg       0.81      0.60      0.67      3000



In [94]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc_tuned))

[[1616 1023]
 [ 177  184]]


Although the overall accuracy of the tuned classifier dropped considerably, the confusion matrix shows a marked improvement in the model’s ability to identify adopted users, accurately classifying over half of them (183 out of 361 in the test set). This improvement suggests that we can now gain meaningful insights into which features are most predictive of future user adoption by examining the features that saw the greatest increase in importance in the tuned model. Let’s dive into those feature importance scores:

In [97]:
df_features['rfc_tuned'] = rfc_tuned.feature_importances_

df_features['difference'] = df_features['rfc_tuned'] - df_features['rfc']

df_features.sort_values('difference', ascending=False)[:15]

,rfc,rfc_tuned,difference
creation_source_PERSONAL_PROJECTS,0.011543,0.080421,0.068877
org_id_0,0.008936,0.065023,0.056087
creation_source_GUEST_INVITE,0.008301,0.063778,0.055477
email_yahoo.com,0.013412,0.044497,0.031085
email_hotmail.com,0.009989,0.037771,0.027782
creation_source_SIGNUP_GOOGLE_AUTH,0.005507,0.022810,0.017304
email_gmail.com,0.016870,0.033124,0.016254
org_id_1,0.007084,0.022089,0.015005
org_id_3,0.006175,0.017236,0.011062
creation_source_ORG_INVITE,0.009691,0.019857,0.010166


The table above indicates that the key factors influencing user adoption are being invited to join another user's workspace or being invited as a guest to an organization. Additionally, membership in certain organizations—particularly 'org_id_0'—increases the likelihood of product adoption. The tuned classifier also highlights the importance of specific email domains (yahoo.com, hotmail.com, and gmail.com) and using Google authentication when signing up. Being invited to become a full member of an organization also significantly boosts the chances of a user adopting the product.
